<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_1000rep_n20_x4_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_1000rep_n20_x4_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    42.830155  38.212730  ...  13708.887486 -2.153310e+06
      1.0    30.006571  29.764006  ...  13939.417267 -4.417214e+05
      2.0    37.302543  42.445116  ...   -442.104981  3.242560e+06
      3.0    11.805456  48.527523  ... -41271.740263  2.421680e+04
      4.0    47.691249  43.989193  ...   8917.328835  3.149649e+06
...                ...        ...  ...           ...           ...
999.0 15.0   44.060313  27.011670  ...  35392.271144  9.792767e+06
      16.0   27.144674  23.135028  ...  22591.789822  9.309978e+04
      17.0   26.622181  19.249270  ...  28997.628808  5.518630e+05
      18.0   47.264025  11.832082  ...  66556.605959 -6.210624e+06
      19.0   47.181299  18.653665  ...  54110.082604  3.983378e+06

[20000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    42.830155  38.212730  ...  13708.887486 -2.153310e+06
      1.0    30.006571  29.764006  ...  13939.417267 -4.417214e+05
      2.0    37.302543  42.445116  ...   -442.104981  3.242560e+06
      3.0    11.805456  48.527523  ... -41271.740263  2.421680e+04
      4.0    47.691249  43.989193  ...   8917.328835  3.149649e+06
...                ...        ...  ...           ...           ...
999.0 15.0   44.060313  27.011670  ...  35392.271144  9.792767e+06
      16.0   27.144674  23.135028  ...  22591.789822  9.309978e+04
      17.0   26.622181  19.249270  ...  28997.628808  5.518630e+05
      18.0   47.264025  11.832082  ...  66556.605959 -6.210624e+06
      19.0   47.181299  18.653665  ...  54110.082604  3.983378e+06

[20000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,42.830155,38.212730,10.819415,-2.140084e+06,-2.139601e+06,13708.887486,-2.153310e+06
1,0.0,1.0,30.006571,29.764006,37.258190,-4.281541e+05,-4.277820e+05,13939.417267,-4.417214e+05
2,0.0,2.0,37.302543,42.445116,22.820808,3.241675e+06,3.242118e+06,-442.104981,3.242560e+06
3,0.0,3.0,11.805456,48.527523,22.173559,-1.724870e+04,-1.705494e+04,-41271.740263,2.421680e+04
4,0.0,4.0,47.691249,43.989193,22.049212,3.158019e+06,3.158567e+06,8917.328835,3.149649e+06
...,...,...,...,...,...,...,...,...,...
19995,999.0,15.0,44.060313,27.011670,22.095554,9.827664e+06,9.828159e+06,35392.271144,9.792767e+06
19996,999.0,16.0,27.144674,23.135028,35.297452,1.153567e+05,1.156916e+05,22591.789822,9.309978e+04
19997,999.0,17.0,26.622181,19.249270,41.933362,5.805283e+05,5.808607e+05,28997.628808,5.518630e+05
19998,999.0,18.0,47.264025,11.832082,45.663953,-6.144602e+06,-6.144067e+06,66556.605959,-6.210624e+06


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    42.830155  38.212730  ...  13708.887486 -2.153310e+06
      1.0    30.006571  29.764006  ...  13939.417267 -4.417214e+05
      2.0    37.302543  42.445116  ...   -442.104981  3.242560e+06
      3.0    11.805456  48.527523  ... -41271.740263  2.421680e+04
      4.0    47.691249  43.989193  ...   8917.328835  3.149649e+06
...                ...        ...  ...           ...           ...
999.0 15.0   44.060313  27.011670  ...  35392.271144  9.792767e+06
      16.0   27.144674  23.135028  ...  22591.789822  9.309978e+04
      17.0   26.622181  19.249270  ...  28997.628808  5.518630e+05
      18.0   47.264025  11.832082  ...  66556.605959 -6.210624e+06
      19.0   47.181299  18.653665  ...  54110.082604  3.983378e+06

[20000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero 

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
0.0,5.0,9.0,628030088722.4375,52791891234307.13,157007522180.60938,6598986404288.392,46.699718676163705,0.0005420368873643167,0.9997289815563178,Reject005=0 : Heteroscedasticity
3.0,8.0,6.0,9573871327855.451,8472009892492.46,1367695903979.35,1694401978498.492,1.179879361562326,0.805183345550653,0.5974083272246735,Reject005=1 : Homoscedasticity
8.0,8.0,6.0,12315664259069.842,37687224191240.625,1759380608438.5488,7537444838248.125,4.08013982826637,0.07135906980189599,0.964320465099052,Reject005=1 : Homoscedasticity
9.0,9.0,5.0,485095860154.0368,25138664253578.605,60636982519.2546,6284666063394.651,93.2796986601225,3.572997295808733e-07,0.9999998213501352,Reject005=0 : Heteroscedasticity
12.0,4.0,10.0,406921116660.1112,146394646595427.53,135640372220.03706,16266071843936.393,143.90469366347142,0.00022880645173795067,0.999885596774131,Reject005=0 : Heteroscedasticity
...,...,...,...,...,...,...,...,...,...,...
992.0,8.0,6.0,59655501821180.34,54345476205201.56,8522214545882.906,10869095241040.312,1.2146513352192105,0.7767385334647148,0.6116307332676426,Reject005=1 : Homoscedasticity
995.0,8.0,6.0,2244079965020.083,44361985875518.16,320582852145.72614,8872397175103.63,26.35793528871041,0.0001490782811217617,0.9999254608594391,Reject005=0 : Heteroscedasticity
996.0,5.0,9.0,781513577718.4967,39156373858434.6,195378394429.62418,4894546732304.325,27.835141515992127,0.0018995853360055381,0.9990502073319972,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,5.0,9.0,628030088722.4375,52791891234307.13,157007522180.60938,6598986404288.392,46.699718676163705,0.0005420368873643167,Reject005=0 : Heteroscedasticity
1.0,10.0,4.0,196882732407842.7,2925273203.3224096,21875859156426.965,975091067.7741365,3.714486749989207e-05,6.62229848935525e-09,Reject005=0 : Heteroscedasticity
2.0,6.0,8.0,10428308500047.105,11037580846839.691,2085661700009.4211,1576797263834.2417,0.7938186365595509,0.7414447261207983,Reject005=1 : Homoscedasticity
3.0,8.0,6.0,9573871327855.451,8472009892492.46,1367695903979.35,1694401978498.492,1.179879361562326,1.194816654449347,Reject005=1 : Homoscedasticity
4.0,10.0,4.0,6454185584475.125,206229229510.35992,717131731608.3472,68743076503.45331,0.07988197225317746,0.026468518205818498,Reject005=0 : Heteroscedasticity
...,...,...,...,...,...,...,...,...,...
995.0,8.0,6.0,2244079965020.083,44361985875518.16,320582852145.72614,8872397175103.63,26.35793528871041,0.0001490782811217617,Reject005=0 : Heteroscedasticity
996.0,5.0,9.0,781513577718.4967,39156373858434.6,195378394429.62418,4894546732304.325,27.835141515992127,0.0018995853360055381,Reject005=0 : Heteroscedasticity
997.0,8.0,6.0,49091471306501.77,70202214417.6508,7013067329500.253,14040442883.53016,0.001906704974728895,1.1610112490439389e-07,Reject005=0 : Heteroscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    632
Reject005=1 : Homoscedasticity      368
Name: Result_test, dtype: int64